In [2]:
import re
import requests
import pandas as pd
import numpy as np
import pymongo
import json
from nltk.corpus import stopwords
from spacy.en import English
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords
from spacy.en import STOP_WORDS
from bs4 import BeautifulSoup
nlp = English()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [3]:
client = pymongo.MongoClient('54.201.199.246', 27016)

wiki_db = client.wikipedia

wiki_col = wiki_db.my_collection

wiki_col.count()

1636

In [4]:
cursor = wiki_col.find()

In [5]:
# next(cursor)['content']

In [6]:
machine_df = pd.DataFrame(list(cursor))

In [7]:
stop = set(stopwords.words('english'))

In [10]:
def cleaner(text):
    text = text.lower()
    text = re.sub('<.{0,3}>',' ',text)
    text = re.sub('{.*\.*}',' ',text)
    text = re.sub('{.*\+.*}',' ',text)
    text = re.sub('[^a-z]',' ',text)
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    text = re.sub('aa','a',text)
    text = re.sub('[0-9]','',text)
    text = re.sub('\s+',' ',text)
    text = ' '.join([i.lemma_ for i in nlp(text) 
                     if i.orth_ not in STOP_WORDS])
    return text

In [8]:
machine_df['content'][0]['*']

'{{Multiple issues|{{refimprove|date=July 2017}}{{more footnotes|date=July 2017}}}}\n\n\'\'\'Data exploration\'\'\' is an approach similar to initial data analysis, whereby a data analyst uses visual exploration to understand what is in a dataset and the characteristics of the data, rather than through traditional data management systems<ref name="Foster">[https://www.fosteropenscience.eu/sites/default/files/pdf/2933.pdf FOSTER Open Science], Overview of Data Exploration Techniques: Stratos Idreos, Olga Papaemmonouil, Surajit Chaudhuri.</ref>. These characteristics can include size or amount of data, completeness of the data, correctness of the data, possible relationships amongst data elements or files/tables in the data.\n\nData exploration is typically conducted using a combination of automated and manual activities.<ref name="Foster" /><ref name="Stanford2011">[http://vis.stanford.edu/files/2011-Wrangler-CHI.pdf Stanford.edu], 2011 Wrangler: Interactive Visual Specification of Data

In [11]:
cleaner(machine_df['content'][0]['*'])

'data exploration approach similar initial datum analysis datum analyst use visual exploration understand dataset characteristic data traditional data management system ref foster https www fosteropenscience eu site default file pdf pdf foster open science overview data exploration technique strato idreos olga papaemmonouil surajit chaudhuri ref characteristic include size datum completeness datum correctness datum possible relationship data element file table datum datum exploration typically conduct combination automate manual activity ref foster ref stanford http vis stanford edu file wrangl chi pdf stanford edu wrangler interactive visual specification data transformation script kandel paepcke hellerstein heer ref automate activity include datum profile datum visualization tabular report analyst initial view datum understanding key characteristic follow manual drill filtering datum identify anomaly pattern identify automated action datum exploration require manual scripting query d

In [24]:
fixed_content = []
for i, item in enumerate(machine_df['content']):
    content = machine_df['content'][i]['*']
    fixed_content.append(content)

In [25]:
machine_df['content'] = fixed_content

In [26]:
machine_df.head()

,_id,article,category,content,page_id
0,5a11baefc7782d026b5037f7,List of datasets for machine learning research,Machine learning,{{Use dmy dates|date=September 2017}}\n{{machi...,49082762
1,5a11baefc7782d026b5037f8,Machine learning,Machine learning,{{for|the journal|Machine Learning (journal)}}...,233488
2,5a11baf0c7782d026b5037f9,Outline of machine learning,Machine learning,<!--... Attention: THIS IS AN OUTLINE\n\n ...,53587467
3,5a11baf0c7782d026b5037fa,Accuracy paradox,Machine learning,The '''accuracy paradox''' for [[predictive an...,3771060
4,5a11baf0c7782d026b5037fb,Action model learning,Machine learning,{{Machine learning bar}}\n\n'''Action model le...,43808044


In [27]:
machine_df['clean_text'] = machine_df['content'].apply(cleaner)

In [34]:
machine_df['clean_text'][0:15]

0     datum set dataset machine learn research cite ...
1     machine learning field computer science give c...
2     attention outline set outline list portal cont...
3     accuracy paradox predictive analytic state pre...
4     action model learning abbreviated action learn...
5     active learning special case semi supervise le...
6     adversarial machine learning research field li...
7     infobox artist aiva nationality luxembourgish ...
8     multiple issue aixi theoretical mathematical f...
9     algorithm selection call instance algorithm se...
10    algorithmic bias occur datum teach machine lea...
11    algorithmic inference gather new development s...
12    file alphago logo reverse svg alphago logo thu...
13    alphago zero version deepmind software alphago...
14    apprenticeship learning apprenticeship inverse...
Name: clean_text, dtype: object